In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder


import warnings
warnings.filterwarnings('ignore')

from scipy.stats import skew
# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm



In [2]:
df = pd.read_csv("datos/bikes_EDA_casual.csv", index_col=0)
df.drop("casual_BOX", inplace=True, axis =1)
df.head(5)

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,hum,windspeed,casual
0,2018-01-01,winter,0,1,1,Monday,0,2,14.110847,80.5833,10.749882,331
1,2018-01-02,winter,0,1,0,Tuesday,1,2,14.902598,69.6087,16.652113,131
2,2018-01-03,winter,0,1,0,Wednesday,1,1,8.050924,43.7273,16.636703,120
3,2018-01-04,winter,0,1,0,Thursday,1,1,8.200000,59.0435,10.739832,108
4,2018-01-05,winter,0,1,0,Friday,1,1,9.305237,43.6957,12.522300,82


In [3]:
lista_categoricas = ['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit', 'casual_BOX']

In [4]:
df_season = df.groupby('season')["casual"].mean().reset_index().sort_values(by = "casual")
df_season

,season,casual
3,winter,336.427778
0,autumn,729.112360
1,spring,1106.097826
2,summer,1202.611702


In [5]:
print(f'winter: 1')
print(f'autumn: {df_season.loc[0, "casual"]/df_season.loc[3, "casual"]}')
print(f'spring: {df_season.loc[1, "casual"]/df_season.loc[3, "casual"]}')
print(f'summer: {df_season.loc[2, "casual"]/df_season.loc[3, "casual"]}')

winter: 1
autumn: 2.167218070893557
spring: 3.2877719949081388
summer: 3.5746504348461565


In [6]:
df_yr = df.groupby('yr')["casual"].mean().reset_index().sort_values(by = "casual")
df_yr

,yr,casual
0,0,677.40274
1,1,1021.09589


In [7]:
print(f'0: 1')
print(f'1: {df_yr.loc[1, "casual"]/df_yr.loc[0, "casual"]}')

0: 1
1: 1.5073690000485334


In [8]:
df_mnth = df.groupby('mnth')["casual"].mean().reset_index().sort_values(by = "casual")
df_mnth

,mnth,casual
0,1,194.225806
1,2,266.035714
11,12,349.887097
10,11,610.050000
2,3,716.838710
9,10,963.870968
3,4,1013.366667
7,8,1161.919355
8,9,1172.050000
4,5,1214.274194


In [9]:
print(f'1: 1')
print(f'2: {df_mnth.loc[1, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'3: {df_mnth.loc[2, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'4: {df_mnth.loc[3, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'5: {df_mnth.loc[4, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'6: {df_mnth.loc[5, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'7: {df_mnth.loc[6, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'8: {df_mnth.loc[7, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'9: {df_mnth.loc[8, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'10: {df_mnth.loc[9, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'11: {df_mnth.loc[10, "casual"]/df_mnth.loc[0, "casual"]}')
print(f'12: {df_mnth.loc[11, "casual"]/df_mnth.loc[0, "casual"]}')

1: 1
2: 1.3697238237596983
3: 3.690749045009135
4: 5.217466644521951
5: 6.25186846038864
6: 6.341931019210541
7: 6.490367048663012
8: 5.982311908320877
9: 6.034471018103305
10: 4.962630792227205
11: 3.1409317389138014
12: 1.8014449427005481


In [10]:
df_holiday = df.groupby('holiday')["casual"].mean().reset_index().sort_values(by = "casual")
df_holiday

,holiday,casual
0,0,838.69209
1,1,1189.00000


In [11]:
print(f'0: 1')
print(f'1: {df_holiday.loc[1, "casual"]/df_holiday.loc[0, "casual"]}')

0: 1
1: 1.417683573764639


In [12]:
df_weekday = df.groupby('weekday')["casual"].mean().reset_index().sort_values(by = "casual")
df_weekday

,weekday,casual
0,Friday,573.701923
4,Thursday,584.134615
6,Wednesday,629.125000
2,Saturday,682.250000
5,Tuesday,959.819048
3,Sunday,1165.211538
1,Monday,1344.676190


In [13]:
print(f'Monday: {df_weekday.loc[1, "casual"]/df_weekday.loc[0, "casual"]}')
print(f'Tuesday: {df_weekday.loc[5, "casual"]/df_weekday.loc[0, "casual"]}')
print(f'Sunday	: {df_weekday.loc[3, "casual"]/df_weekday.loc[0, "casual"]}')
print(f'Wednesday: {df_weekday.loc[6, "casual"]/df_weekday.loc[0, "casual"]}')
print(f'Saturday: {df_weekday.loc[2, "casual"]/df_weekday.loc[0, "casual"]}')
print(f'Friday: 1')
print(f'Thursday: {df_weekday.loc[4, "casual"]/df_weekday.loc[0, "casual"]}')

Monday: 2.3438586073832868
Tuesday: 1.6730274189622216
Sunday	: 2.0310399731836086
Wednesday: 1.0966060504483366
Saturday: 1.1892064024134752
Friday: 1
Thursday: 1.0181848654990362


In [14]:
df_weathersit = df.groupby('weathersit')["casual"].mean().reset_index().sort_values(by = "casual")
df_weathersit

,weathersit,casual
2,3,185.476190
1,2,689.882114
0,1,964.030238


In [15]:
print(f'3: 1')
print(f'2: {df_weathersit.loc[1, "casual"]/df_weathersit.loc[2, "casual"]}')
print(f'1: {df_weathersit.loc[0, "casual"]/df_weathersit.loc[2, "casual"]}')

3: 1
2: 3.719518457058768
1: 5.197595632657475


In [16]:
df_workingday = df.groupby('workingday')["casual"].mean().reset_index().sort_values(by = "casual")
df_workingday

,workingday,casual
1,1,803.167331
0,0,950.710526


In [17]:
print(f'1: 1')
print(f'0: {df_workingday.loc[0, "casual"]/df_workingday.loc[1, "casual"]}')

1: 1
0: 1.1837016895521375


In [18]:
df["season_encoding"] = df["season"].map({"winter": 1, "autumn": 2.167, "spring": 3.288, "summer": 3.575})
df['yr_encoding'] = df['yr'].map({0:1, 1:1.507})
df['mnth_encoding'] = df['mnth'].map({1:1, 2: 1.370, 3: 3.691, 4:5.217, 5:6.252, 6:6.342, 7: 6.490, 8:5.982, 9:6.034, 10:4.963, 11:3.141, 12:1.801})
df['holiday_encoding'] = df['holiday'].map({0:1, 1:1.418})
df['weekday_encoding'] = df['weekday'].map({'Monday':2.344, 'Tuesday':1.673, 'Wednesday':1.097, 'Thursday':1.018, 'Friday':1, 'Saturday':1.189, 'Sunday':2.031})
df['weathersit_encoding'] = df['weathersit'].map({1:5.198, 2:3.720, 3:1})
df['workingday_encoding'] = df['workingday'].map({0:1.184, 1:1})

In [19]:
df.drop(columns=['dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit'], inplace=True)

In [20]:
X = df.drop("casual", axis = 1)
y = df["casual"]

In [21]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 23)

In [22]:
arbol = DecisionTreeRegressor(random_state = 23)

arbol.fit(x_train, y_train)

DecisionTreeRegressor(random_state=23)

In [23]:
max_features = np.sqrt(len(x_train.columns))
max_features

3.1622776601683795

In [24]:
print(arbol.tree_.max_depth)

21


In [25]:
y_pred_test_dt = arbol.predict(x_test)
y_pred_train_dt = arbol.predict(x_train)

In [26]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [27]:
dt_results1 = metricas(y_test, y_train, y_pred_test_dt, y_pred_train_dt, "Decision Tree I")
dt_results1

,MAE,MSE,RMSE,R2,set,modelo
0,302.253425,251974.89726,501.971012,0.411565,test,Decision Tree I
1,0.000000,0.00000,0.000000,1.000000,train,Decision Tree I


In [28]:
param = {"max_depth": [2, 3, 4, 5, 6, 7, 8, 9, 10], 
        "max_features": [2,3,4,],
        "min_samples_split": [10, 20, 30, 50, 150, 200],
        "min_samples_leaf": [10, 20, 30, 50, 150, 200]} 

In [29]:
gs = GridSearchCV(
            estimator=DecisionTreeRegressor(random_state = 23), 
            param_grid= param, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [30]:
gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(random_state=23),
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'max_features': [2, 3, 4],
                         'min_samples_leaf': [10, 20, 30, 50, 150, 200],
                         'min_samples_split': [10, 20, 30, 50, 150, 200]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [31]:
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=8, max_features=4, min_samples_leaf=20,
                      min_samples_split=10, random_state=23)

In [32]:
y_pred_test_dt2 = mejor_modelo.predict(x_test)
y_pred_train_dt2 = mejor_modelo.predict(x_train)

In [33]:
dt_results2 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "Decision tree II")
dt_results2

,MAE,MSE,RMSE,R2,set,modelo
0,302.468360,229848.648165,479.425331,0.463236,test,Decision tree II
1,255.683313,145722.414357,381.736053,0.696836,train,Decision tree II


In [34]:
df_decision_results = pd.concat([dt_results1, dt_results2], axis = 0)
df_decision_results

,MAE,MSE,RMSE,R2,set,modelo
0,302.253425,251974.897260,501.971012,0.411565,test,Decision Tree I
1,0.000000,0.000000,0.000000,1.000000,train,Decision Tree I
0,302.468360,229848.648165,479.425331,0.463236,test,Decision tree II
1,255.683313,145722.414357,381.736053,0.696836,train,Decision tree II


In [35]:
param2 = {"max_depth": [ 5, 6, 7, 8], 
        "max_features": [2,3,4,5,6,7],
        "min_samples_split": [5, 10, 20, 50],
        "min_samples_leaf": [5, 10, 20, 50]} 

In [36]:
gs2 = GridSearchCV(
            estimator=DecisionTreeRegressor(random_state = 23), 
            param_grid= param2, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [37]:
gs2.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(random_state=23),
             param_grid={'max_depth': [5, 6, 7, 8],
                         'max_features': [2, 3, 4, 5, 6, 7],
                         'min_samples_leaf': [5, 10, 20, 50],
                         'min_samples_split': [5, 10, 20, 50]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [38]:
mejor_modelo2 = gs2.best_estimator_
mejor_modelo2

DecisionTreeRegressor(max_depth=7, max_features=7, min_samples_leaf=5,
                      min_samples_split=5, random_state=23)

In [39]:
y_pred_test_dt3 = mejor_modelo2.predict(x_test)
y_pred_train_dt3 = mejor_modelo2.predict(x_train)

In [40]:
dt_results3 = metricas(y_test, y_train, y_pred_test_dt3, y_pred_train_dt3, "Decision tree III")
dt_results3

,MAE,MSE,RMSE,R2,set,modelo
0,290.090580,200309.711562,447.559730,0.532218,test,Decision tree III
1,174.456182,68450.202831,261.629897,0.857595,train,Decision tree III


In [41]:
df_decision_results = pd.concat([df_decision_results, dt_results3], axis = 0)
df_decision_results

,MAE,MSE,RMSE,R2,set,modelo
0,302.253425,251974.897260,501.971012,0.411565,test,Decision Tree I
1,0.000000,0.000000,0.000000,1.000000,train,Decision Tree I
0,302.468360,229848.648165,479.425331,0.463236,test,Decision tree II
1,255.683313,145722.414357,381.736053,0.696836,train,Decision tree II
0,290.090580,200309.711562,447.559730,0.532218,test,Decision tree III
1,174.456182,68450.202831,261.629897,0.857595,train,Decision tree III


In [42]:
bosque = RandomForestRegressor(max_depth=5, max_features=7, min_samples_leaf=5,
                      min_samples_split=20, random_state = 23)
bosque.fit(x_train, y_train)

RandomForestRegressor(max_depth=5, max_features=7, min_samples_leaf=5,
                      min_samples_split=20, random_state=23)

In [43]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [44]:
rf_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest")
rf_results

,MAE,MSE,RMSE,R2,set,modelo
0,250.543309,156509.899412,395.613321,0.634504,test,Random Forest
1,187.231005,76950.958956,277.400359,0.839909,train,Random Forest


In [48]:
param_rf = {"n_estimators": [250,500],
        "max_depth": [ 5, 6, 7, 8], 
        "max_features": [3,4,5,6, 7],
        "min_samples_split": [10, 20, 50, 70],
        "min_samples_leaf": [5, 10, 20, 30]}

In [49]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(random_state=23), 
            param_grid= param_rf, 
            cv=10, 
            verbose=-1, 
            return_train_score = True, 
            scoring="neg_mean_squared_error") 

In [50]:
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(random_state=23),
             param_grid={'max_depth': [5, 6, 7, 8],
                         'max_features': [3, 4, 5, 6, 7],
                         'min_samples_leaf': [5, 10, 20, 30],
                         'min_samples_split': [10, 20, 50, 70],
                         'n_estimators': [250, 500]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [51]:
bosque2 = gs_rf.best_estimator_
bosque2

RandomForestRegressor(max_depth=8, max_features=7, min_samples_leaf=5,
                      min_samples_split=10, n_estimators=500, random_state=23)

In [52]:
y_pred_test_rf2 = bosque2.predict(x_test)
y_pred_train_rf2 = bosque2.predict(x_train)

In [53]:
dt_results_rf2 = metricas(y_test, y_train, y_pred_test_rf2, y_pred_train_rf2, "Random Forest II")
dt_results_rf2

,MAE,MSE,RMSE,R2,set,modelo
0,233.916825,143930.834709,379.382175,0.663880,test,Random Forest II
1,148.227329,52602.025938,229.351316,0.890565,train,Random Forest II
